# Dataset Collection

Configure the parameters in the next cell before starting the dataset collection.

In [ ]:
# Number of datapoints that will be collected.
DATASET_SIZE = 2000
# Relative path of directory at which the dataset will be saved. Must end with "/".
DATASET_PATH = "dataset/"
# IP adress of device running mobile application.
APPLICATION_HOST = "192.168.1.101"
# Port on which the server of the application is listening.
APPLICATION_PORT = 1280
# IP adress of device capturing images.
CAMERA_HOST = "192.168.1.106"
# Port on which the server for capturing is listening.
CAMERA_PORT = 1300

In [ ]:
import json
import socket
import socketserver
import os
import base64
from io import BytesIO
from PIL import Image
from IPython.display import clear_output
import time

class Dataset:
    def __init__(self, size: int, path: str):
        self.total_size = size
        self.current_size = 0
        self.path = path
        
    def getImageFromServer(self) -> dict:
        # Retry if connection timed out or did not succeed.
        # Either no response or wrong response or network error.
        for i in range(100):
            try:
                client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
                client.settimeout(0.2)
                
                client.connect((CAMERA_HOST, CAMERA_PORT))
                request = {"header": "image"}
                
                # Send new line at the end to enable the other side
                # to directly receive the request by reading one line of input.
                client.sendall((json.dumps(request) + "\n").encode())
                response = client.makefile().readline()
                return response

            except:
                print("Connection for image refused: Attempt", i+1)
                continue
            
    def getMaskFromServer(self) -> dict:
        for i in range(100):
            try:
                client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
                client.settimeout(0.2)

                client.connect((CAMERA_HOST, CAMERA_PORT))
                request = {"header": "mask"}

                # Send new line at the end to enable the other side
                # to directly receive the request by reading one line of input.
                client.sendall((json.dumps(request) + "\n").encode())
                response = client.makefile().readline()
                return response

            except:
                print("Connection for mask refused: Attempt", i+1)
                continue
        
    def getFromServerAndSaveImage(self, path: str):
        # Conversion from base64 format to png image.
        image_base64 = self.getImageFromServer()
        image_bytes = base64.b64decode(image_base64)
        image_file = BytesIO(image_bytes)
        image = Image.open(image_file)

        image.save(path)
        
    def getFromServerAndSaveMask(self, path: str):
        # Conversion from base64 format to png image.
        image_base64 = self.getMaskFromServer()
        image_bytes = base64.b64decode(image_base64)
        image_file = BytesIO(image_bytes)
        image = Image.open(image_file)

        image.save(path)
        
    def saveAnnotations(self, path: str, data: dict):
        try:
            with open(path, "w") as file:
                file.write(json.dumps(data, indent=2))
        except Exception as e:
            print(e.message, e.args)
    
    '''
    Handles sending of requests to the mobile application.
    '''
    def makeRequest(self, header: str) -> dict:
        for i in range(100):
            try:
                client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
                client.connect((APPLICATION_HOST, APPLICATION_PORT))
                client.settimeout(0.1)

                request = {"header": header}
                
                # Send new line at the end to enable the other side
                # to directly receive the request by reading one line of input.
                client.sendall((json.dumps(request) + "\n").encode())

                response = json.loads(client.makefile().readline())

                client.close()

                return response
            except:
                print("Connection to application refused: Attempt", i+1)
                continue

dataset = Dataset(DATASET_SIZE, DATASET_PATH)

In [ ]:
while(dataset.current_size < dataset.total_size):
    response = dataset.makeRequest("ready?")
    
    # Retry until expected response is returned.
    if(response["header"] != "ready!"):
        continue
    
    # Wait to fix issue with blurred camera image if image is taken to quick after GUI change.
    time.sleep(0.1)
    dataset.getFromServerAndSaveImage(dataset.path + f"{dataset.current_size}.png")

    response = dataset.makeRequest("classes?")
    if(response["header"] != "classes!"):
        continue
    dataset.saveAnnotations(response["body"])

    for mask_index in response["body"]:
        response = dataset.makeRequest("next state?")
        time.sleep(0.15)
        dataset.getFromServerAndSaveMask(dataset.path + f"{dataset.current_size}_{mask_index}.png")

    response = dataset.makeRequest("randomize?")

    dataset.current_size = dataset.current_size + 1
    print("Datapoint", dataset.current_size, "/", dataset.total_size, "collected.")